In [30]:
from glob import glob
import shutil
import os

files = glob('*/**/*.*', recursive = True)
files.extend(glob('*'))
files = [f for f in files if f.endswith('pdf') or f.endswith('doc') or f.endswith('docx') or f.endswith('xlsx') or f.endswith('pptx') or f.endswith('PDF') or f.endswith('ppt')]
len(files)

16237

In [31]:
for f in files:
    if f.startswith('pdf/'):
        continue
    try:
        shutil.move(f, os.path.join('pdf', os.path.split(f)[1]))
    except:
        pass

In [34]:
# !zip -r bumigemilang.com-pdf.zip pdf

In [36]:
files = glob('pdf/*')
len(files)

16245

In [37]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
from tika import parser
from bs4.element import Comment
from bs4 import BeautifulSoup
from glob import glob

In [38]:
TIKA_HOST = 'http://localhost:9998'

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body, is_html=True):
    if is_html:
        soup = BeautifulSoup(body, 'html.parser')
    else:
        soup = body
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return '\n'.join(t.strip() for t in visible_texts).strip()


def parse_tika(file, minlen=2):
    raw_xml = parser.from_file(file, TIKA_HOST, xmlContent=True, requestOptions = {'timeout': 3600})
    body = BeautifulSoup(raw_xml['content']).find('body')
    t = []
    for b in body:
        try:
            t_ = text_from_html(b, is_html=False)
            if len(t_) > minlen:
                t.append(t_)
        except Exception as e:
            pass
    
    return t

In [40]:
with open('files.jsonl', 'w') as fopen:
    for f in tqdm(files):
        try:
            t = '\n'.join(parse_tika(f))
            data = {
                'file': f,
                'text': t,
            }
            fopen.write(f'{json.dumps(data)}\n')
        except Exception as e:
            print(e)